In [5]:
import math

class Value():
    def __init__(self, data, _children=(), _op=''):
        assert isinstance(data, (int, float)), "Values can only be ints/floats"
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op

    def __repr__(self):
        return f"Value(data={self.data})"
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward

        return out
    
    def __radd__(self, other):
        return self + other
    
    def __neg__(self):
        return -1 * self
    
    def __sub__(self, other):
        return self + (-other)
    
    def __rsub__(self, other):
        return other + (-self)
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out
    
    def __rmul__(self, other):
        return self * other
    
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "Powers can only be ints/floats as of now"
        out = Value(self.data**other, (self, ), f'**{other}')

        def _backward():
            self.grad += other* self.data**(other-1) * out.grad

        out._backward = _backward
        return out
    
    def __truediv__(self, other):
        return self * other**-1

    def __rtruediv__(self, other):
        return other * self**-1

    def exp(self):
        out = Value(math.exp(self.data)), (self, ), 'exp'

        def _backward():
            self.grad += out.data * out.grad

        out._backward = _backward
        return out
    
    def tanh(self):
        x = self.data
        t = (math.exp(2*x)-1)/(math.exp(2*x)+1)
        out = Value(t, (self, ), 'tanh')

        def _backward():
            self.grad += (1 - t**2) * out.grad

        out._backward = _backward
        return out

    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        self.grad = 1.0
        for n in reversed(topo):
            n._backward()

In [6]:
a = Value(-4.0)
b = Value(2.0)
c = a + b
d = a * b + b**3
c += c + 1
c += 1 + c + (-a)
d += d * 2 + (b + a)
d += 3 * d + (b - a)
e = c - d
f = e**2
g = f / 2.0
g += 10.0 / f
print(f'{g.data:.4f}') # prints 24.7041, the outcome of this forward pass
g.backward()
print(f'{a.grad:.4f}') # prints 138.8338, i.e. the numerical value of dg/da
print(f'{b.grad:.4f}') # prints 645.5773, i.e. the numerical value of dg/db

10.5000
456.0000
1843.0000


In [2]:
import random

class Neuron():
    def __init__(self, nin: int):
        self.weights = [Value(random.uniform(0,1)) for _ in range(nin)]
        self.bias = Value(random.uniform(0,1))

    def __call__(self, x):
        act = sum((wi*xi for wi, xi in zip(self.weights, x)), self.bias) # Add up the dot product of each weight and each input and add the bias (wx + b)
        out = act.tanh()
        return out
    
    def parameters(self):
        return self.weights + [self.bias]
    
class Layer():
    def __init__(self, nin:int, n:int):
        self.neurons = [Neuron(nin) for _ in range(n)]
    
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]
    
class MLP():
    def __init__(self, nin:int, nouts: list[int]):
        size = [nin] + nouts
        self.layers = [Layer(size[i], size[i+1]) for i in range(len(nouts))]
    
    def __call__(self,x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for layer in self.layers for neuron in layer.neurons for p in neuron.parameters()]
    

In [353]:
n = MLP(3, [4,4,1])

In [320]:
def train_loop(nn: MLP, lr:float, xs, y, epoch):
    ypreds = [nn(x) for x in xs]
    loss = sum((yout - ygt)**2 for ygt, yout in zip(y, ypreds))

    # Optimizer Zero Grad
    for p in nn.parameters():
        p.grad = 0.0

    # Loss Backward
    loss.backward()

    # Optimizer Step
    for p in nn.parameters():
        p.data += -lr * p.grad
    
    print(f"Epoch: {epoch+1} | Loss: {loss.data: .5f} | Preds: {ypreds}")

In [355]:
xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]

ytrue = [1.0, -1.0, -1.0, 1.0]
epochs = 200
lr = 0.1

for epoch in range(epochs):
    # lr = .1 - 0.9*epoch/100
    # if epoch % 20 == 0:
    #     print(lr)
    train_loop(n, lr, xs, ytrue, epoch)

Epoch: 1 | Loss:  0.00123 | Preds: [Value(data=0.9796626274738738), Value(data=-0.9851126493085889), Value(data=-0.9797288385632503), Value(data=0.9863985125400929)]
Epoch: 2 | Loss:  0.00122 | Preds: [Value(data=0.9797235945295683), Value(data=-0.9851561680326982), Value(data=-0.9797894762974432), Value(data=0.9864366445060513)]
Epoch: 3 | Loss:  0.00122 | Preds: [Value(data=0.9797840677638001), Value(data=-0.9851993176945186), Value(data=-0.9798495936039462), Value(data=0.9864744642395603)]
Epoch: 4 | Loss:  0.00121 | Preds: [Value(data=0.9798440535523708), Value(data=-0.9852421034320112), Value(data=-0.9799091978004137), Value(data=0.9865119759113435)]
Epoch: 5 | Loss:  0.00120 | Preds: [Value(data=0.979903558159049), Value(data=-0.9852845302839611), Value(data=-0.9799682960619461), Value(data=0.9865491836154248)]
Epoch: 6 | Loss:  0.00120 | Preds: [Value(data=0.979962587738049), Value(data=-0.9853266031924195), Value(data=-0.9800268954246326), Value(data=0.9865860913709168)]
Epoch:

In [367]:
from torchvision import transforms
import torch

from PIL import Image

from pathlib import Path

def path_to_tensor(path: Path):
    img = Image.open(path)
    img_tensor = transforms.ToTensor()(img)
    return img_tensor

In [366]:
type(path_to_tensor('./charlie.jpg'))

torch.Tensor

In [ ]:
class ConvNet(MLP):
    def __init__(self, kernel_size: int, stride: int, padding:int, nin: int, nouts: int):
        super.__init__()
        

    def __call__(self, x: torch.Tensor):
        

SyntaxError: incomplete input (1311994476.py, line 6)